In [ ]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

# wczytuję dane
df_january = pq.read_table(source='/content/yellow_tripdata_2023-01.parquet').to_pandas()


# zmieniam daty na datetime w pandas
df_january['tpep_pickup_datetime'] = pd.to_datetime(df_january['tpep_pickup_datetime'], format='%y-%m-%d %H:%M:%S')
df_january['tpep_dropoff_datetime'] = pd.to_datetime(df_january['tpep_dropoff_datetime'], format='%y-%m-%d %H:%M:%S')

# dzielę kolumny z czasem począku podrózy na rok, mieiąc, dzień, dzień tygodnia, godzinę

df_january['year'] = pd.to_datetime(df_january['tpep_pickup_datetime']).dt.year
df_january['month'] = pd.to_datetime(df_january['tpep_pickup_datetime']).dt.month
df_january['day'] = pd.to_datetime(df_january['tpep_pickup_datetime']).dt.day
df_january['hour_of_day'] = pd.to_datetime(df_january['tpep_pickup_datetime']).dt.hour

# obliczam czas podróży
pickup_array = df_january['tpep_pickup_datetime'].values
dropoff_array = df_january['tpep_dropoff_datetime'].values

trip_duration = np.subtract(dropoff_array, pickup_array)

# dodaję nową kolumnę trip_duration do df i zamieniam na sekundy
df_january['trip_duration'] = pd.Series(trip_duration)

df_january['trip_duration'] = df_january['trip_duration'].dt.total_seconds()

# tworzę nowy dataframe z wybranym tygodniem - 22 - 29 maja
df1 = df_january[df_january['month'] == 1]
df2 = df1[df1['day'] >= 22]
df = df2[df2['day'] <= 29]

# czyszczę dane - wyrzucam niepotrzebna kolumny, takie jak 'tpep_pickup_datetime', 'tpep_dropoff_datetime' i 'VendorID'
df.drop(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)

# czyszczę dane - mui być przynjamniej jeden pasażer, przejazd muiał być dłuższy niz 0
df = df[df['passenger_count'] >= 1]
df = df[df['trip_duration'] > 0]

# zapiuję w ten sposob otrzymany dataframe do csc
df.to_csv('taxi_january.csv')

<ipython-input-7-020f325ecacd>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)
